In [ ]:
"""
================================================================================
COMPLETE NOTEBOOK 3: INFERENCE & EXPLAINABILITY (LIME)
================================================================================
This notebook:
1. Loads the best trained model
2. Provides inference functions for new text
3. Generates LIME explanations
4. Performs similarity search using FAISS
"""

# ============================================================================
# SETUP
# ============================================================================

import pandas as pd
import numpy as np
import joblib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Sentence embeddings
from sentence_transformers import SentenceTransformer

# FAISS
import faiss

# LIME for explainability
import lime
from lime.lime_text import LimeTextExplainer

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("="*80)
print("MBIC BIAS DETECTION - COMPLETE PIPELINE")
print("Notebook 3: Inference & Explainability")
print("="*80)
print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

In [ ]:
# ============================================================================
# STEP 1: LOAD ALL ARTIFACTS
# ============================================================================

print("\n" + "="*80)
print("STEP 1: LOADING ARTIFACTS")
print("="*80)

# Load model
best_model = joblib.load('/content/best_model_final.pkl')
print("✓ Loaded: best_model_final.pkl")

# Load metadata
metadata = joblib.load('/content/model_metadata.pkl')
print("✓ Loaded: model_metadata.pkl")
print(f"  Model: {metadata['model_name']}")
print(f"  Test F1 Macro: {metadata['test_f1_macro']:.4f}")

# Load label encoder
label_encoder = joblib.load('/content/label_encoder.pkl')
print("✓ Loaded: label_encoder.pkl")

# Load feature scaler
feature_scaler = joblib.load('/content/feature_scaler.pkl')
print("✓ Loaded: feature_scaler.pkl")

# Load linguistic extractor
ling_extractor = joblib.load('/content/linguistic_extractor.pkl')
print("✓ Loaded: linguistic_extractor.pkl")

# Load FAISS index
faiss_index = faiss.read_index('/content/mbic_faiss_hybrid.index')
print(f"✓ Loaded: mbic_faiss_hybrid.index ({faiss_index.ntotal} vectors)")

# Load processed dataset (for similarity search)
processed_df = pd.read_excel('/content/processed_dataset.xlsx')
print(f"✓ Loaded: processed_dataset.xlsx ({len(processed_df)} samples)")

# Load embedding models
print("\n Loading embedding models...")
model_mpnet = SentenceTransformer('all-mpnet-base-v2')
model_minilm = SentenceTransformer('all-MiniLM-L6-v2')
print("✓ Embedding models loaded")

In [ ]:
# ============================================================================
# STEP 2: CREATE INFERENCE PIPELINE
# ============================================================================

print("\n" + "="*80)
print("STEP 2: CREATING INFERENCE PIPELINE")
print("="*80)

class BiasDetectionPipeline:
    """Complete inference pipeline"""

    def __init__(self, model, label_encoder, model_mpnet, model_minilm,
                 ling_extractor, feature_scaler):
        self.model = model
        self.label_encoder = label_encoder
        self.model_mpnet = model_mpnet
        self.model_minilm = model_minilm
        self.ling_extractor = ling_extractor
        self.feature_scaler = feature_scaler

    def extract_features(self, text):
        """Extract hybrid features for a single text"""
        # Embeddings
        emb_mpnet = self.model_mpnet.encode([text])[0]
        emb_minilm = self.model_minilm.encode([text])[0]
        embeddings = np.concatenate([emb_mpnet, emb_minilm])

        # Linguistic features
        ling_features = self.ling_extractor.extract(text)
        ling_array = np.array(list(ling_features.values())).reshape(1, -1)
        ling_scaled = self.feature_scaler.transform(ling_array).flatten()

        # Combine
        features = np.concatenate([embeddings, ling_scaled])
        return features

    def predict(self, text):
        """Predict bias label for text"""
        features = self.extract_features(text).reshape(1, -1)

        # Handle different model types
        prediction = self.model.predict(features)

        # Convert to label if needed
        if isinstance(prediction[0], (int, np.integer)):
            label = self.label_encoder.inverse_transform(prediction)[0]
        else:
            label = prediction[0]

        return label

    def predict_proba(self, text):
        """Get prediction probabilities if available"""
        features = self.extract_features(text).reshape(1, -1)

        # Check if model has predict_proba
        if hasattr(self.model, 'predict_proba'):
            proba = self.model.predict_proba(features)[0]
            return {label: prob for label, prob in zip(self.label_encoder.classes_, proba)}
        elif hasattr(self.model, 'calibrated_classifier'):
            # For confidence-based classifier
            proba = self.model.calibrated_classifier.predict_proba(features)[0]
            # Map binary proba to three classes
            return {
                'Biased': proba[1],
                'Non-biased': proba[0],
                'No agreement': 1.0 - proba.max()
            }
        else:
            return None

    def predict_batch(self, texts):
        """Predict for multiple texts"""
        return [self.predict(text) for text in texts]

# Create pipeline instance
pipeline = BiasDetectionPipeline(
    model=best_model,
    label_encoder=label_encoder,
    model_mpnet=model_mpnet,
    model_minilm=model_minilm,
    ling_extractor=ling_extractor,
    feature_scaler=feature_scaler
)

print("✓ Inference pipeline created")

In [ ]:
# ============================================================================
# STEP 3: TEST INFERENCE ON EXAMPLES
# ============================================================================

print("\n" + "="*80)
print("STEP 3: TESTING INFERENCE")
print("="*80)

# Test examples
test_examples = [
    "The politician was caught red-handed in a massive corruption scandal.",
    "The study was conducted using standard scientific methodology.",
    "Some people believe the earth is flat, but this claim has been debunked.",
    "All immigrants are criminals and should be deported immediately.",
    "The data shows a correlation between the two variables.",
]

print("\nTest predictions:")
print("-" * 80)

for i, text in enumerate(test_examples, 1):
    prediction = pipeline.predict(text)
    proba = pipeline.predict_proba(text)

    print(f"\n{i}. Text: {text}")
    print(f"   Prediction: {prediction}")
    if proba:
        print(f"   Probabilities:")
        for label, prob in sorted(proba.items(), key=lambda x: x[1], reverse=True):
            print(f"     {label:15s}: {prob:.4f}")

In [ ]:
# ============================================================================
# STEP 4: SIMILARITY SEARCH WITH FAISS
# ============================================================================

print("\n" + "="*80)
print("STEP 4: SIMILARITY SEARCH")
print("="*80)

def find_similar_samples(query_text, k=5):
    """Find k most similar samples in the dataset"""
    # Extract features for query
    query_features = pipeline.extract_features(query_text).reshape(1, -1)

    # Search FAISS index
    distances, indices = faiss_index.search(query_features.astype('float32'), k)

    results = []
    for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
        results.append({
            'rank': i + 1,
            'index': idx,
            'distance': dist,
            'similarity': 1 / (1 + dist),  # Convert distance to similarity
            'text': processed_df.iloc[idx]['sentence'],
            'label': processed_df.iloc[idx]['Label_bias']
        })

    return results

# Test similarity search
print("\nTesting similarity search...")
query = "The corrupt politician stole millions from taxpayers."
print(f"\nQuery: {query}")
print(f"Prediction: {pipeline.predict(query)}\n")

similar = find_similar_samples(query, k=5)
print("Top 5 similar samples:")
print("-" * 80)

for result in similar:
    print(f"\n{result['rank']}. Similarity: {result['similarity']:.4f}")
    print(f"   Label: {result['label']}")
    print(f"   Text: {result['text'][:100]}...")

In [ ]:
# ============================================================================
# STEP 5: LIME EXPLAINABILITY
# ============================================================================

print("\n" + "="*80)
print("STEP 5: LIME EXPLAINABILITY")
print("="*80)

# Create LIME explainer
class_names = list(label_encoder.classes_)

# Wrapper function for LIME (must return probabilities)
def predict_fn_for_lime(texts):
    """Wrapper for LIME that returns probability matrix"""
    results = []

    for text in texts:
        proba = pipeline.predict_proba(text)

        if proba:
            # Convert dict to array in correct order
            proba_array = [proba.get(label, 0.0) for label in class_names]
        else:
            # If no probabilities, create one-hot encoding
            pred = pipeline.predict(text)
            proba_array = [1.0 if label == pred else 0.0 for label in class_names]

        results.append(proba_array)

    return np.array(results)

# Create explainer
print("\nInitializing LIME explainer...")
explainer = LimeTextExplainer(
    class_names=class_names,
    random_state=RANDOM_STATE
)
print("✓ LIME explainer ready")

def explain_prediction(text, num_features=10):
    """Generate LIME explanation for a prediction"""
    print(f"\n{'='*80}")
    print("LIME EXPLANATION")
    print(f"{'='*80}")
    print(f"\nText: {text}")

    # Get prediction
    prediction = pipeline.predict(text)
    proba = pipeline.predict_proba(text)

    print(f"\nPrediction: {prediction}")
    if proba:
        print("Probabilities:")
        for label, prob in sorted(proba.items(), key=lambda x: x[1], reverse=True):
            print(f"  {label:15s}: {prob:.4f}")

    # Generate explanation
    print(f"\nGenerating LIME explanation...")
    exp = explainer.explain_instance(
        text,
        predict_fn_for_lime,
        num_features=num_features,
        num_samples=500
    )

    # Show explanation
    print(f"\n{'='*80}")
    print("FEATURE IMPORTANCE")
    print(f"{'='*80}")

    # Get prediction class index
    pred_idx = list(class_names).index(prediction) if prediction in class_names else 0

    # Show top features
    print(f"\nTop {num_features} features for prediction '{prediction}':")
    print("-" * 60)
    print(f"{'Feature':<30} {'Weight':<15} {'Impact'}")
    print("-" * 60)

    for feature, weight in exp.as_list(label=pred_idx):
        impact = "Supports ✓" if weight > 0 else "Against ✗"
        print(f"{feature:<30} {weight:>+.4f}        {impact}")

    return exp

In [ ]:
# Test LIME on examples
print("\n" + "="*80)
print("GENERATING EXPLANATIONS FOR TEST EXAMPLES")
print("="*80)

# Example 1: Clearly biased
exp1 = explain_prediction(
    "All politicians are corrupt liars who only care about themselves.",
    num_features=8
)

# Example 2: Neutral
exp2 = explain_prediction(
    "The committee met yesterday to discuss the proposed legislation.",
    num_features=8
)

# Example 3: Ambiguous
exp3 = explain_prediction(
    "Some experts believe this policy might have unintended consequences.",
    num_features=8
)

In [ ]:
# ============================================================================
# STEP 6: CREATE INTERACTIVE INFERENCE FUNCTION
# ============================================================================

print("\n" + "="*80)
print("STEP 6: INTERACTIVE INFERENCE FUNCTION")
print("="*80)

def analyze_text_complete(text, show_similar=True, show_explanation=True):
    """
    Complete analysis of a text:
    - Prediction with probabilities
    - Similar samples from dataset
    - LIME explanation
    """
    print("\n" + "="*80)
    print("COMPLETE BIAS ANALYSIS")
    print("="*80)
    print(f"\nInput: {text}")

    # Prediction
    print("\n" + "-"*80)
    print("1. PREDICTION")
    print("-"*80)
    prediction = pipeline.predict(text)
    proba = pipeline.predict_proba(text)

    print(f"\nPredicted Label: {prediction}")
    if proba:
        print("\nConfidence Scores:")
        for label, prob in sorted(proba.items(), key=lambda x: x[1], reverse=True):
            bar = "█" * int(prob * 40)
            print(f"  {label:15s}: {prob:.4f} {bar}")

    # Similar samples
    if show_similar:
        print("\n" + "-"*80)
        print("2. SIMILAR SAMPLES FROM DATASET")
        print("-"*80)
        similar = find_similar_samples(text, k=3)
        for result in similar:
            print(f"\n  {result['rank']}. Similarity: {result['similarity']:.4f} | Label: {result['label']}")
            print(f"     {result['text'][:80]}...")

    # LIME explanation
    if show_explanation:
        print("\n" + "-"*80)
        print("3. FEATURE IMPORTANCE (LIME)")
        print("-"*80)
        exp = explainer.explain_instance(
            text,
            predict_fn_for_lime,
            num_features=6,
            num_samples=300
        )

        pred_idx = list(class_names).index(prediction) if prediction in class_names else 0
        print(f"\nKey words influencing '{prediction}' prediction:")
        for feature, weight in exp.as_list(label=pred_idx)[:6]:
            impact = "+" if weight > 0 else "-"
            print(f"  {impact} {feature:<25} (weight: {weight:>+.4f})")

    print("\n" + "="*80)

# Example usage
print("\n✓ Interactive function ready. Example usage:")
print("\n  analyze_text_complete('Your text here')")

# Test it
analyze_text_complete(
    "This radical extremist policy will destroy our economy and lead to chaos.",
    show_similar=True,
    show_explanation=True
)

In [ ]:

# ============================================================================
# STEP 7: BATCH INFERENCE FUNCTION
# ============================================================================

print("\n" + "="*80)
print("STEP 7: BATCH INFERENCE")
print("="*80)

def predict_batch_with_details(texts):
    """Predict multiple texts and return detailed results"""
    results = []

    for i, text in enumerate(texts):
        prediction = pipeline.predict(text)
        proba = pipeline.predict_proba(text)

        result = {
            'index': i,
            'text': text,
            'prediction': prediction,
        }

        if proba:
            result.update({
                'prob_biased': proba.get('Biased', 0.0),
                'prob_non_biased': proba.get('Non-biased', 0.0),
                'prob_no_agreement': proba.get('No agreement', 0.0),
                'confidence': max(proba.values())
            })

        results.append(result)

    return pd.DataFrame(results)

# Test batch inference
batch_texts = [
    "The research methodology followed established protocols.",
    "These corrupt officials are destroying our country!",
    "The data suggests a possible correlation.",
    "All members of that group are dangerous criminals.",
    "The committee will review the proposal next week."
]

print("\nBatch inference example:")
results_df = predict_batch_with_details(batch_texts)
print("\n", results_df[['text', 'prediction', 'confidence']].to_string(index=False))

In [ ]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("NOTEBOOK 3 COMPLETE - SUMMARY")
print("="*80)

print(f"\n✅ INFERENCE PIPELINE READY")

print(f"\n📊 Available Functions:")
print(f"  1. pipeline.predict(text) - Single prediction")
print(f"  2. pipeline.predict_proba(text) - With probabilities")
print(f"  3. pipeline.predict_batch(texts) - Batch prediction")
print(f"  4. find_similar_samples(text, k=5) - Find similar samples")
print(f"  5. explain_prediction(text) - LIME explanation")
print(f"  6. analyze_text_complete(text) - Complete analysis")
print(f"  7. predict_batch_with_details(texts) - Batch with details")

print(f"\n💡 USAGE EXAMPLES:")
print(f"""
# Single prediction
pred = pipeline.predict("Your text here")

# With probabilities
proba = pipeline.predict_proba("Your text here")

# Complete analysis
analyze_text_complete("Your text here")

# Batch processing
df = pd.DataFrame({{'text': your_texts}})
df['prediction'] = pipeline.predict_batch(df['text'].tolist())
""")

print(f"\n🎯 MODEL PERFORMANCE:")
print(f"  Model: {metadata['model_name']}")
print(f"  Test F1 Macro: {metadata['test_f1_macro']:.4f}")
print(f"  Test Accuracy: {metadata['test_accuracy']:.4f}")

print("\n" + "="*80)
print("ALL NOTEBOOKS COMPLETE - SYSTEM READY FOR DEPLOYMENT")
print("="*80)
print(f"\nCompleted at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)